In [1]:
from typing import *

# Part 1

The small crab challenges you to a game! The crab is going to mix up some cups, and you have to predict where they'll end up.

The cups will be arranged in a circle and labeled clockwise (your puzzle input). For example, if your labeling were 32415, there would be five cups in the circle; going clockwise around the circle from the first cup, the cups would be labeled 3, 2, 4, 1, 5, and then back to 3 again.

Before the crab starts, it will designate the first cup in your list as the current cup. The crab is then going to do 100 moves.

Each move, the crab does the following actions:

The crab picks up the three cups that are immediately clockwise of the current cup. They are removed from the circle; cup spacing is adjusted as necessary to maintain the circle.
The crab selects a destination cup: the cup with a label equal to the current cup's label minus one. If this would select one of the cups that was just picked up, the crab will keep subtracting one until it finds a cup that wasn't just picked up. If at any point in this process the value goes below the lowest value on any cup's label, it wraps around to the highest value on any cup's label instead.
The crab places the cups it just picked up so that they are immediately clockwise of the destination cup. They keep the same order as when they were picked up.
The crab selects a new current cup: the cup which is immediately clockwise of the current cup.
For example, suppose your cup labeling were 389125467. If the crab were to do merely 10 moves, the following changes would occur:

```
-- move 1 --
cups: (3) 8  9  1  2  5  4  6  7 
pick up: 8, 9, 1
destination: 2

-- move 2 --
cups:  3 (2) 8  9  1  5  4  6  7 
pick up: 8, 9, 1
destination: 7

-- move 3 --
cups:  3  2 (5) 4  6  7  8  9  1 
pick up: 4, 6, 7
destination: 3

-- move 4 --
cups:  7  2  5 (8) 9  1  3  4  6 
pick up: 9, 1, 3
destination: 7

-- move 5 --
cups:  3  2  5  8 (4) 6  7  9  1 
pick up: 6, 7, 9
destination: 3

-- move 6 --
cups:  9  2  5  8  4 (1) 3  6  7 
pick up: 3, 6, 7
destination: 9

-- move 7 --
cups:  7  2  5  8  4  1 (9) 3  6 
pick up: 3, 6, 7
destination: 8

-- move 8 --
cups:  8  3  6  7  4  1  9 (2) 5 
pick up: 5, 8, 3
destination: 1

-- move 9 --
cups:  7  4  1  5  8  3  9  2 (6)
pick up: 7, 4, 1
destination: 5

-- move 10 --
cups: (5) 7  4  1  8  3  9  2  6 
pick up: 7, 4, 1
destination: 3

-- final --
cups:  5 (8) 3  7  4  1  9  2  6 
```
In the above example, the cups' values are the labels as they appear moving clockwise around the circle; the current cup is marked with ( ).

After the crab is done, what order will the cups be in? Starting after the cup labeled 1, collect the other cups' labels clockwise into a single string with no extra characters; each number except 1 should appear exactly once. In the above example, after 10 moves, the cups clockwise from 1 are labeled 9, 2, 6, 5, and so on, producing 92658374. If the crab were to complete all 100 moves, the order after cup 1 would be 67384529.

Using your labeling, simulate 100 moves. What are the labels on the cups after cup 1?

In [4]:
raw_input = "562893147"

In [103]:
from __future__ import annotations
from collections import deque
from itertools import islice

class Game:
    def __init__(self, cups: List[int]):
        self.d = deque(cups)
        self.min_cup_value = min(cups)
        self.max_cup_value = max(cups)

    def step(self):
        current_cup = self.d.popleft()
        next_cups = list(reversed([self.d.popleft() for _ in range(3)]))
        dest_idx = self._find_destination_idx(current_cup, next_cups)
        self.d.rotate(-dest_idx-1)
        self.d.extendleft(next_cups)
        self.d.rotate(dest_idx+1)
        self.d.append(current_cup)
        
        #print(f"{current_cup=}, {next_cups=}, {dest_idx=}, {self.d=}")
        
        
    def play(self, n_turns: int = 100):
        for _ in range(n_turns):
            self.step()
        print(self.order_after_1)

    def _find_destination_idx(self, current_cup, next_cups):
        destination_cup = current_cup - 1
        while True:
            #print(f"{current_cup=}, {destination_cup=}, {list(next_cups)=}, {self.d=}")
            if destination_cup not in list(next_cups) and destination_cup > 0:
                return self.d.index(destination_cup)
            elif destination_cup <= self.min_cup_value:
                destination_cup = self.max_cup_value
            else:
                destination_cup -= 1
    
    @property
    def order_after_1(self) -> str:
        idx_1 = self.d.index(1)
        self.d.rotate(-idx_1)
        out = "".join(map(str, self.d))[1:]
        self.d.rotate(idx_1)
        return out
        
    
    @classmethod
    def from_raw(cls, s: str) -> Game:
        cups = [int(i) for i in s]
        return cls(cups)


In [104]:
g = Game.from_raw("562893147")

In [105]:
g.play()

38925764


# Part 2

Due to what you can only assume is a mistranslation (you're not exactly fluent in Crab), you are quite surprised when the crab starts arranging many cups in a circle on your raft - one million (1000000) in total.

Your labeling is still correct for the first few cups; after that, the remaining cups are just numbered in an increasing fashion starting from the number after the highest number in your list and proceeding one by one until one million is reached. (For example, if your labeling were 54321, the cups would be numbered 5, 4, 3, 2, 1, and then start counting up from 6 until one million is reached.) In this way, every number from one through one million is used exactly once.

After discovering where you made the mistake in translating Crab Numbers, you realize the small crab isn't going to do merely 100 moves; the crab is going to do ten million (10000000) moves!

The crab is going to hide your stars - one each - under the two cups that will end up immediately clockwise of cup 1. You can have them if you predict what the labels on those cups will be when the crab is finished.

In the above example (389125467), this would be 934001 and then 159792; multiplying these together produces 149245887792.

Determine which two cups will end up immediately clockwise of cup 1. What do you get if you multiply their labels together?



In [399]:
from itertools import chain

class Game2:
    def __init__(self, cup_to_next_cup: Iterable[int]):
        self.capacity = 1_000_001
        self.storage = self.capacity * [None]
        for c,nc in cup_to_next_cup:
            self.storage[c] = nc
        
            
        self.min_cup_value = 1
        self.max_cup_value = 1_000_000
        self.pntr = 1_000_000
        
    def step(self):
        # grap the cup we're at
        current_cup = self.storage[self.pntr]
        # find next cups
        next_cups = self._next_cups(current_cup)
        # take them out 
        self.storage[current_cup] = self.storage[next_cups[2]]
        # get the destination cup
        destination_cup = self._find_destination_cup(current_cup, next_cups)
        # insert our cups back in 
        self.storage[next_cups[2]] = self.storage[destination_cup]
        self.storage[destination_cup] = next_cups[0]

        # make sure we grab the next cup next time
        self.pntr = current_cup

    def play(self, n_turns: int = 100):
        for n in range(n_turns):
            if n%1_000_000 == 0:
                print(f"On turn {n}")
            self.step()
        print(self.order_after_1)

    def _next_cups(self, current_cup):
        out = []
        for _ in range(3):
            current_cup = self.storage[current_cup]
            out.append(current_cup)
        return out
    
    def _find_destination_cup(self, current_cup, next_cups):
        destination_cup = current_cup - 1
        while True:
            #print(f"{current_cup=}, {destination_cup=}, {list(next_cups)=}, {self.storage=}")
            if destination_cup not in next_cups and destination_cup > 0:
                return destination_cup
            elif destination_cup <= self.min_cup_value:
                destination_cup = self.max_cup_value
            else:
                destination_cup -= 1
    
    @property
    def order_after_1(self) -> str:
        x = self.storage[1]
        y = self.storage[x]
        return x*y        
    
    @classmethod
    def from_raw(cls, s: str) -> Game:
        full_input = chain(s, range(len(s)+1, 1_000_001))
        offset_input = chain(s[1:], range(len(s)+1, 1_000_001), s[0])
        cup_to_next_cup = ((int(c), int(nc)) for c, nc in zip(full_input, offset_input))
        return cls(cup_to_next_cup)


In [397]:
g2 = Game2.from_raw(raw_input)

In [398]:
g2.play(10_000_000)

On turn 0
On turn 1000000
On turn 2000000
On turn 3000000
On turn 4000000
On turn 5000000
On turn 6000000
On turn 7000000
On turn 8000000
On turn 9000000
131152940564
